# OmniSafe Tutorial - CLI Command

OmniSafe: https://github.com/PKU-MARL/omnisafe

Documentation: https://omnisafe.readthedocs.io/en/latest/

Safety-Gymnasium: https://www.safety-gymnasium.com/

[Safety-Gymnasium](https://www.safety-gymnasium.com/) is a highly scalable and customizable Safe Reinforcement Learning library, aiming to deliver a good view of benchmarking Safe Reinforcement Learning (Safe RL) algorithms and a more standardized setting of environments. 

## Introduction

In this section, you will be introduced to the CLI tool of OmniSafe, which allows you to completely avoid dealing with code. You can call OmniSafe through command-line commands, even if you have no knowledge of programming. With the information presented in this section, you can easily become proficient in using OmniSafe.

## Install OmniSafe and dependencies using Pip


```
pip install omnisafe
```

## Install OmniSafe and dependencies from source

```
# Clone the repo
git clone https://github.com/PKU-MARL/omnisafe
cd omnisafe

# Create a conda environment
conda create -n omnisafe python=3.8
conda activate omnisafe

# Install omnisafe
pip install -e .
```

In [ ]:
!pip install omnisafe

In [ ]:
# Clone the repo
!git clone https://github.com/PKU-MARL/omnisafe
%cd omnisafe

# Create a conda environment
!conda create -n omnisafe python=3.8
!conda activate omnisafe

# Install omnisafe
!pip install -e .

## CLI Usage Examples

### Quick experiment

The `train` command aims to provide you with a quick interface to run experiments.

By using the following command, you can start training with default parameters, and conveniently modify some common parameters through the command line.

In [ ]:
!omnisafe train --algo PPO --total-steps 2048

If you want to further understand the usage of the `train` command, simply use `--help`。



In [ ]:
!omnisafe train --help

For uncommon parameters, we also support modifications. However, this goes beyond the original intention of the `train` command, which was designed for quick and easy experimentation. Therefore, if there is a need to modify a large number of uncommon parameters, please refer to the `train-config` section below.

In the example below, we modified the value of `update_cycle` belonging to `algo_cfgs` in the default parameters via the command line.

**Please note**:

+ For nested parameter key-value pairs, use a colon `:` to connect them.
+ To modify a parameter, the corresponding key-value pair must exist in the previously defined `YAML` file.
+ To save the YAML file that contains the default parameters for each algorithm in OmniSafe, please refer to [this link](https://github.com/PKU-MARL/omnisafe/tree/main/omnisafe/configs).
+ To modify uncommon parameters, use `--custom-cfgs`. For this, you must provide the key-value pairs according to their respective relationships and precede each string with the `--custom-cfgs` flag. For example: `--custom-cfgs key1 --custom-cfgs value1 --custom-cfgs key2 --custom-cfgs value2`.

In [ ]:
!omnisafe train --algo PPOLag --env-id SafetyHumanoidVelocity-v4 --total-steps 2048 --custom-cfgs algo_cfgs:update_cycle --custom-cfgs 1024

### Configure your experiments

The `train-config` command reads the parameters you provide from the YAML file path you specify and incrementally updates them on top of the default parameters.

With the `train-config` command, you can provide more detailed and specific parameters without having to deal with a huge number of parameters on the command line, which can be prone to errors.

**Please note**:

+ You need to create your own parameter file, which must follow the same format as the default parameters. For algorithms supported in the library, you can refer to the file format [here](https://github.com/PKU-MARL/omnisafe/tree/main/omnisafe/configs).
+ The configuration file can be created in any location, just provide its path when calling the `train-config` command on the command line. We recommend that you define it in the current working directory because any files generated during the execution of your command will also be saved here.
Here is an example where we will run the previous experiment again in a different way.


In [ ]:
!mkdir ./save_config
%cd ./save_config

Next, define your experiment parameters.

In [ ]:
%%bash

echo "algo:
  PPOLag
env_id:
  SafetyAntVelocity-v4
train_cfgs:
  total_steps:
    1024
  vector_env_nums: 1
algo_cfgs:
  update_cycle:
    1024" > config.yaml

Next, simply specify the directory of the configuration file for the `train-config` command.

In [ ]:
!omnisafe train-config ./config.yaml

Similarly, you can use the `--help` option to view more related information.

In [ ]:
!omnisafe train-config --help

### Render and evaluate your experiments


In the CLI, we also provide support for visualizing training data through the `eval` command. The `eval` command supports flexible invocation with multiple parameters, and you can get detailed explanations by using the `--help` option.

In [ ]:
!omnisafe eval --help



Next, we will perform rendering and evaluation on the previous experiment run, and the visual results will be saved in the same directory as the experiment data.

**Please note**:

+ The `--no-render` option can be used to turn off visualization.

In [ ]:
%ls
%cd ./train_dict
%ls

**Please note**: 

+ Rendering may encounter problems on servers without a display. Please try the following steps. If you encounter any problems on your own machine, please create an [issue](https://github.com/PKU-MARL/omnisafe/issues?q=is%3Aissue+is%3Aopen+sort%3Aupdated-desc) on GitHub for OmniSafe to let us help you resolve the issue.

In [ ]:
%%bash
apt-get install libosmesa6-dev
apt-get install python3-opengl

In [ ]:
%env MUJOCO_GL=osmesa
%env PYOPENGL_PLATFORM=osmesa

env: MUJOCO_GL=osmesa
env: PYOPENGL_PLATFORM=osmesa


In [ ]:
!omnisafe eval ./PPOLag-{SafetyAntVelocity-v4} --num-episode 1

Let's check the generated video.

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

# Please fill the path of folder containing your video which is shown above here
show_videos(video_path='')

### Quick benchmarking and large-scale experiment



`experiment_grid` is an incredibly useful tool. With it, you can quickly test all feasible parameter combinations that influence algorithm performance. Typically, `experiment_grid` is only needed when a significant number of parameters must be specified, and as such, it can only be invoked through a configuration file. Please refer to [this link](https://github.com/PKU-MARL/omnisafe/blob/main/examples/benchmarks/example_cli_benchmark_config.yaml) for the configuration file format.

In essence, you must specify the key-value pairs of the parameters, and if nested key-value pairs are present, use a colon : to separate them. All feasible values must be enclosed in square brackets `[]`, and indentation must adhere to the YAML file format requirements.

**Please note**:

+ We recommend that you set the `num-pool` value to a number that evenly divides the total number of experiments, ensuring that the number of tasks executed at any given time is roughly the same during the entire experiment process.
+ `exp_name` specifies the directory in which all of your experiments for this run will be located. To prevent file conflicts between two experiments, you must explicitly provide a value for `exp_name` each time you run `experiment_grid`.


In the OmniSafe CLI tool, you can invoke `experiment_grid` using the `benchmark` command.


Certainly, you can also obtain further usage instructions by using the `--help` option.

In [ ]:
!omnisafe benchmark --help

Next, let's run a set of experiments using the `benchmark` command. This will make your work quite efficient when a large number of parameters need to be adjusted.

In [ ]:
%%bash

echo "algo:
  ['PolicyGradient', 'NaturalPG']
env_id:
  ['SafetyAntVelocity-v4']
logger_cfgs:use_wandb:
  [False]
train_cfgs:vector_env_nums:
  [2]
train_cfgs:torch_threads:
  [1]
train_cfgs:total_steps:
  4096
algo_cfgs:update_cycle:
  2048
seed:
  [0]" > grid_config.yaml

To run the command, you must:

1. Provide an experiment name.
2. Provide the maximum number of processes that can be run simultaneously in this experiment.
3. Provide the path to the configuration file containing the parameter settings.




In [ ]:
!omnisafe benchmark test_grid 2 ./grid_config.yaml

### Quick analyzing the results



In the `train` and `train-config` commands, you can directly specify whether to plot the training curve, visualize the model during the process, or evaluate the model's performance using the `--plot`, `--render`, and `--evaluate` options.

However, in `benchmark`, due to the significantly greater complexity of the analysis compared to a single experiment, we have introduced a separate `analyze-grid` command to analyze the results.

**Please note**:

+ `render` and `evaluate` are indistinguishable to the model during the process, as they both involve testing the model in different environment instances. You can think of rendering as a form of evaluating that simultaneously generates a video.
+ During testing, the condition `deterministic==True` applies to the network.




Next, we will introduce the `analyze-grid` command. By using the `--help` option, you can see the following parameters that can be specified:

+ `path`: indicates the path where a set of `experiment_grid` experiments are saved. As long as there is a complete experiment folder started by `experiment_grid` under the given directory, the program will automatically recursively search for its specific location.

+ `parameter`: specifies the name of the parameter whose effect you want to analyze through performance comparison. If there are nested parameters, please use : to separate them.

+ `compare_num`: indicates the maximum number of different parameter values you want to see on the same graph when analyzing the effect of the parameter. OmniSafe will automatically generate all such combinations. For example, if there are three algorithms and only one value for each of the other parameters, and `parameter='algo' compare_num=2`, you will get three graphs, each comparing the performance of two algorithms under the current environment and parameters. This can greatly reduce your mental burden during comparison.

+ `cost_limit`: the black line displayed in the cost part of the performance graph, indicating the value of the `cost_limit` that should be followed.

You may have noticed that in the previous section, users can also specify the `values` option to select specific parameter values for comparison. Due to the limitations of command line operations, we do not support this option here. If you need to use it, you can create a Python script to call the statistics tools to implement it.

Next, we will analyze the experiment that was run above.

In [ ]:
!omnisafe analyze-grid --help

Regarding the specific usage of the data analysis tools, a more detailed introduction was given in the first section. Here, we only show how to call them through CLI.

To generate performance comparison graphs for PPO and PolicyGradient in different environments in the above experiment grid, you can use the following CLI command:

In [ ]:
!omnisafe analyze-grid ./benchmark/test_grid algo --compare-num 2

### Explore
OmniSafe is a young and constantly growing library, and we will continue to improve its functionality and optimize the user experience. This section aims to cover the core usage of OmniSafe's CLI tools. You can look forward to new interactive ways and features in the future. If you have any better insights during use, please feel free to contact us. We will carefully listen to your suggestions.

By using `--help`, you can continue exploring OmniSafe.

In [ ]:
!omnisafe --help